In [1]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [2]:
import time

In [3]:
from pymongo import MongoClient

In [4]:
import pandas as pd

In [5]:
client = MongoClient("mongodb://mongo1:49005,mongo2:49006,mongo3:49007/?replicaSet=myReplicaSet")

In [6]:
df = pd.read_csv('test_data_mongo.csv')
df

,dish_name,price,category
0,Пельмени,2000,первое блюдо
1,Борщ,1500,первое блюдо
2,Суп-харчо,1800,первое блюдо
3,Стейк,4000,второе блюдо
4,Салат Цезарь,2500,второе блюдо
5,Картошка фри,1200,десерт
6,Тирамису,3000,десерт
7,Кока-кола,1000,напиток
8,Кофе,1500,напиток
9,Мороженое,1800,десерт


In [6]:
database = client["Restaurant"]
database

Database(MongoClient(host=['mongo1:49005', 'mongo3:49007', 'mongo2:49006'], document_class=dict, tz_aware=False, connect=True, replicaset='myReplicaSet'), 'Restaurant')

In [7]:
str(client.list_database_names())

"['Restaurant', 'admin', 'config', 'local']"

In [8]:
collection = database["Menu"]
collection

Collection(Database(MongoClient(host=['mongo1:49005', 'mongo3:49007', 'mongo2:49006'], document_class=dict, tz_aware=False, connect=True, replicaset='myReplicaSet'), 'Restaurant'), 'Menu')

In [11]:
collection.insert_many(df.to_dict("records"))

InsertManyResult([ObjectId('6608349f412872baa84d47b1'), ObjectId('6608349f412872baa84d47b2'), ObjectId('6608349f412872baa84d47b3'), ObjectId('6608349f412872baa84d47b4'), ObjectId('6608349f412872baa84d47b5'), ObjectId('6608349f412872baa84d47b6'), ObjectId('6608349f412872baa84d47b7'), ObjectId('6608349f412872baa84d47b8'), ObjectId('6608349f412872baa84d47b9'), ObjectId('6608349f412872baa84d47ba')], acknowledged=True)

In [9]:
import random
import string
from faker import Faker

In [10]:
def generate_random_data(num_records):
    fake = Faker()
    categories = ["первое блюдо", "второе блюдо", "напитки", "десерт"]
    
    data = []
    for _ in range(num_records):
        dish_name = fake.text(max_nb_chars=30)
        price = random.randint(1000, 5000)
        category = random.choice(categories)
        data.append([dish_name, price, category])
    
    return data

In [11]:
# Создание DataFrame из сгенерированных данных
def create_dataframe(num_records):
    data = generate_random_data(num_records)
    df = pd.DataFrame(data, columns=["dish_name", "price", "category"])
    return df

In [12]:
# Генерация записей и создание DataFrame
df = create_dataframe(2000000)

# Вывод сгенерированных данных
print(df)

                             dish_name  price      category
0          Half central party address.   1374        десерт
1              Subject line mind hour.   3622  второе блюдо
2        Responsibility fast four put.   3628       напитки
3         Wrong tonight by choice him.   4572       напитки
4                 Summer truth entire.   2486        десерт
...                                ...    ...           ...
1999995  Author would car moment lose.   4142  первое блюдо
1999996     Site good prepare brother.   4946        десерт
1999997        Stay accept difference.   1284       напитки
1999998   Charge save story financial.   4678       напитки
1999999               Local have upon.   3469  первое блюдо

[2000000 rows x 3 columns]


In [13]:
start = time.time()
collection.insert_many(df.to_dict("records"))
end = time.time() - start

In [14]:
print(f"Время заполнения коллекции: {end / 60} minutes")

Время заполнения коллекции: 0.8229571064313252 minutes


In [15]:
# Выборка всех записей из коллекции
start = time.time()
result = collection.find({})
end = time.time() - start

In [16]:
print(f"Время выполнения запроса: {end / 60 } minutes")

Время выполнения запроса: 0.0 minutes


In [17]:
# Вывод всех записей
for document in result:
    print(document)

{'_id': ObjectId('661d7e7e9bcfa8de695671f5'), 'dish_name': 'Half central party address.', 'price': 1374, 'category': 'десерт'}
{'_id': ObjectId('661d7e7e9bcfa8de695671f6'), 'dish_name': 'Subject line mind hour.', 'price': 3622, 'category': 'второе блюдо'}
{'_id': ObjectId('661d7e7e9bcfa8de695671f7'), 'dish_name': 'Responsibility fast four put.', 'price': 3628, 'category': 'напитки'}
{'_id': ObjectId('661d7e7e9bcfa8de695671f8'), 'dish_name': 'Wrong tonight by choice him.', 'price': 4572, 'category': 'напитки'}
{'_id': ObjectId('661d7e7e9bcfa8de695671f9'), 'dish_name': 'Summer truth entire.', 'price': 2486, 'category': 'десерт'}
{'_id': ObjectId('661d7e7e9bcfa8de695671fa'), 'dish_name': 'Tell part her ground.', 'price': 4736, 'category': 'десерт'}
{'_id': ObjectId('661d7e7e9bcfa8de695671fb'), 'dish_name': 'Feel role hard task.', 'price': 4160, 'category': 'первое блюдо'}
{'_id': ObjectId('661d7e7e9bcfa8de695671fc'), 'dish_name': 'Already laugh soldier lawyer.', 'price': 3377, 'category':

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [18]:
def get_dishes_by_category(collection, category):
    query = {"category": category}
    result = collection.find(query)
    return result

In [19]:
def get_dishes_below_price(collection, price_threshold):
    query = {"price": {"$lt": price_threshold}}
    result = collection.find(query)
    return result

In [20]:
def get_dishes_in_price_range(collection, min_price, max_price):
    query = {"price": {"$gte": min_price, "$lte": max_price}}
    result = collection.find(query)
    return result

In [21]:
# Получение всех блюд определенной категории
category = "первое блюдо"
start = time.time()
dishes_by_category = get_dishes_by_category(collection, category)
end = time.time() - start
print(f"All dishes in category '{category}':")
for dish in dishes_by_category:
    print(dish)

All dishes in category 'первое блюдо':
{'_id': ObjectId('661d7e7e9bcfa8de695671fb'), 'dish_name': 'Feel role hard task.', 'price': 4160, 'category': 'первое блюдо'}
{'_id': ObjectId('661d7e7e9bcfa8de69567203'), 'dish_name': 'Green share seem himself.', 'price': 4010, 'category': 'первое блюдо'}
{'_id': ObjectId('661d7e7e9bcfa8de69567205'), 'dish_name': 'Sign use federal affect.', 'price': 4251, 'category': 'первое блюдо'}
{'_id': ObjectId('661d7e7e9bcfa8de6956720f'), 'dish_name': 'Popular read both.', 'price': 2484, 'category': 'первое блюдо'}
{'_id': ObjectId('661d7e7e9bcfa8de69567211'), 'dish_name': 'Control local call economy.', 'price': 1728, 'category': 'первое блюдо'}
{'_id': ObjectId('661d7e7e9bcfa8de69567213'), 'dish_name': 'Prevent that newspaper.', 'price': 2239, 'category': 'первое блюдо'}
{'_id': ObjectId('661d7e7e9bcfa8de69567215'), 'dish_name': 'Strong nation fish candidate.', 'price': 2891, 'category': 'первое блюдо'}
{'_id': ObjectId('661d7e7e9bcfa8de69567217'), 'dish_n

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [22]:
print(f"Время выполнения запроса: {end * 1000} ms")

Время выполнения запроса: 1.0039806365966797 ms


In [23]:
# Получение всех блюд с ценой ниже заданного порога
price_threshold = 3000
start = time.time()
dishes_below_price = get_dishes_below_price(collection, price_threshold)
end = time.time() - start
print(f"All dishes below price {price_threshold}:")
for dish in dishes_below_price:
    print(dish)

All dishes below price 3000:
{'_id': ObjectId('661d7e7e9bcfa8de695671f5'), 'dish_name': 'Half central party address.', 'price': 1374, 'category': 'десерт'}
{'_id': ObjectId('661d7e7e9bcfa8de695671f9'), 'dish_name': 'Summer truth entire.', 'price': 2486, 'category': 'десерт'}
{'_id': ObjectId('661d7e7e9bcfa8de695671fe'), 'dish_name': 'View relate one sister none.', 'price': 1042, 'category': 'напитки'}
{'_id': ObjectId('661d7e7e9bcfa8de695671ff'), 'dish_name': 'Score house call long coach.', 'price': 2774, 'category': 'напитки'}
{'_id': ObjectId('661d7e7e9bcfa8de69567200'), 'dish_name': 'Lot tax water person social.', 'price': 2795, 'category': 'второе блюдо'}
{'_id': ObjectId('661d7e7e9bcfa8de69567201'), 'dish_name': 'As apply college traditional.', 'price': 1248, 'category': 'десерт'}
{'_id': ObjectId('661d7e7e9bcfa8de69567202'), 'dish_name': 'Have bank his.', 'price': 1363, 'category': 'десерт'}
{'_id': ObjectId('661d7e7e9bcfa8de69567206'), 'dish_name': 'Black new guy system.', 'pric

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'_id': ObjectId('661d7e839bcfa8de6974b817'), 'dish_name': 'Budget add my economic fish.', 'price': 1732, 'category': 'первое блюдо'}
{'_id': ObjectId('661d7e839bcfa8de6974b81a'), 'dish_name': 'Machine area send good tough.', 'price': 1113, 'category': 'первое блюдо'}
{'_id': ObjectId('661d7e839bcfa8de6974b81b'), 'dish_name': 'Those everyone me.', 'price': 1796, 'category': 'напитки'}
{'_id': ObjectId('661d7e839bcfa8de6974b81c'), 'dish_name': 'Enjoy financial type.', 'price': 1245, 'category': 'напитки'}
{'_id': ObjectId('661d7e839bcfa8de6974b81d'), 'dish_name': 'Ten success north notice.', 'price': 1182, 'category': 'второе блюдо'}
{'_id': ObjectId('661d7e839bcfa8de6974b821'), 'dish_name': 'Many team black benefit car.', 'price': 2700, 'category': 'второе блюдо'}
{'_id': ObjectId('661d7e839bcfa8de6974b823'), 'dish_name': 'Agent toward about official.', 'price': 2697, 'category': 'второе блюдо'}
{'_id': ObjectId('661d7e839bcfa8de6974b829'), 'dish_name': 'Mouth beyond explain set.', 'pr

In [24]:
print(f"Время выполнения запроса: {end * 1000} ms")

Время выполнения запроса: 0.0 ms


In [25]:
# Получение всех блюд с ценой в диапазоне между заданными значениями
min_price = 2000
max_price = 4000
start = time.time()
dishes_in_price_range = get_dishes_in_price_range(collection, min_price, max_price)
end = time.time() - start
print(f"All dishes in price range {min_price}-{max_price}:")
for dish in dishes_in_price_range:
    print(dish)

All dishes in price range 2000-4000:
{'_id': ObjectId('661d7e7e9bcfa8de695671f6'), 'dish_name': 'Subject line mind hour.', 'price': 3622, 'category': 'второе блюдо'}
{'_id': ObjectId('661d7e7e9bcfa8de695671f7'), 'dish_name': 'Responsibility fast four put.', 'price': 3628, 'category': 'напитки'}
{'_id': ObjectId('661d7e7e9bcfa8de695671f9'), 'dish_name': 'Summer truth entire.', 'price': 2486, 'category': 'десерт'}
{'_id': ObjectId('661d7e7e9bcfa8de695671fc'), 'dish_name': 'Already laugh soldier lawyer.', 'price': 3377, 'category': 'второе блюдо'}
{'_id': ObjectId('661d7e7e9bcfa8de695671fd'), 'dish_name': 'Real certainly nearly enjoy.', 'price': 3329, 'category': 'напитки'}
{'_id': ObjectId('661d7e7e9bcfa8de695671ff'), 'dish_name': 'Score house call long coach.', 'price': 2774, 'category': 'напитки'}
{'_id': ObjectId('661d7e7e9bcfa8de69567200'), 'dish_name': 'Lot tax water person social.', 'price': 2795, 'category': 'второе блюдо'}
{'_id': ObjectId('661d7e7e9bcfa8de69567204'), 'dish_name'

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'_id': ObjectId('661d7e839bcfa8de69749743'), 'dish_name': 'Wait line no seem message.', 'price': 2384, 'category': 'второе блюдо'}
{'_id': ObjectId('661d7e839bcfa8de69749745'), 'dish_name': 'He Mr oil address.', 'price': 3166, 'category': 'первое блюдо'}
{'_id': ObjectId('661d7e839bcfa8de69749748'), 'dish_name': 'Activity push approach in.', 'price': 2983, 'category': 'десерт'}
{'_id': ObjectId('661d7e839bcfa8de69749749'), 'dish_name': 'Tend society dream maintain.', 'price': 2791, 'category': 'второе блюдо'}
{'_id': ObjectId('661d7e839bcfa8de6974974b'), 'dish_name': 'Garden bad down.', 'price': 3446, 'category': 'второе блюдо'}
{'_id': ObjectId('661d7e839bcfa8de6974974d'), 'dish_name': 'Item point full.', 'price': 3428, 'category': 'второе блюдо'}
{'_id': ObjectId('661d7e839bcfa8de6974974e'), 'dish_name': 'Set course since page so.', 'price': 3512, 'category': 'второе блюдо'}
{'_id': ObjectId('661d7e839bcfa8de69749750'), 'dish_name': 'The policy give end.', 'price': 3536, 'category':

In [26]:
print(f"Время выполнения запроса: {end * 1000} ms")

Время выполнения запроса: 0.0 ms


Удалить записи:

In [29]:
def delete_all_records(collection):
    result = collection.delete_many({})
    print(f"Удалено {result.deleted_count} записей.")

In [30]:
delete_all_records(collection)

Удалено 2000000 записей.
